Importing data handling libraies


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.tokenize import  sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df=pd.read_csv('/content/final.csv')                               #readind the csv file created before


In [ ]:
df.head()

,Unnamed: 0,URL_ID,Title,Contents
0,0,123.0,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn..."
1,1,321.0,Rise of e-health and its impact on humans by t...,"\nThe rise of e-health, or the use of electron..."
2,2,4321.0,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac..."
3,3,432.0,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac..."
4,4,2893.8,Rise of Chatbots and its impact on customer su...,\nThe human race is known to come up with inve...


In [ ]:
sentences = []                                           #counting the number of sentences in each articles
for article in df['Contents']:
  sentences.append(len(sent_tokenize(article)))

In [ ]:
words = []                                               #counting the number of words in each articles
for article in df['Contents']:
  words.append(len(word_tokenize(article)))

In [ ]:
ps=PorterStemmer()                                #initializing the port stemmer, which helps in educes the words to the root word

Cleaning the contents of articles


In [ ]:
Corpus=[]
for i in range(len(df['Contents'])):
  msg=re.sub('[^a-zA-Z]',' ', df['Contents'][i])      #removing anything beside lower a to lower z and Capital A to capital Z
  msg=msg.lower()                                     # converting all the words to lower case
  msg=msg.split(" ")
  msg=[ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]           #removing stopwards for english language and appling stemming
  msg=' '.join(msg)
  Corpus.append(msg)

In [ ]:
words_cleaned = []
for article in Corpus:
  words_cleaned.append(len(word_tokenize(article)))


In [ ]:
words_cleaned[0]

1005

analyzation the content of articles

In [ ]:
positive_word=[]                                                      #creating a list of positive words reviced from the drive
p = open("positive-words.txt", "r")
for x in p:
  positive_word.append(x.lower().replace('\n',''))

In [ ]:
Negative_word=[]

In [ ]:
N = open("negative-words.txt", "r",encoding='latin-1')                #creating a list of negative  words reviced from the drive
for x in N:
  Negative_word.append(x.lower().replace('\n',''))


In [ ]:
positive_score = [0]*len(df)                                         #initializing of postive score of thes size of dataframe
for i in range(len(df['Contents'])):
  for word in positive_word:
    for letter in Corpus[i].lower().split(' '):
      if letter==word:
        positive_score[i]+=1

In [ ]:
Negative_score = [0]*len(df)                                         #initializing of negative score of thes size of dataframe
for i in range(len(df['Contents'])):
  for word in Negative_word:
    for letter in Corpus[i].lower().split(' '):
      if letter==word:
        Negative_score[i]+=1

In [ ]:
df['POSITIVE SCORE'] = positive_score                                #adding the list to dataframe
df['NEGATIVE SCORE'] = Negative_score

In [ ]:
df.head()

,Unnamed: 0,URL_ID,Title,Contents,POSITIVE SCORE,NEGATIVE SCORE
0,0,123.0,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn...",84,15
1,1,321.0,Rise of e-health and its impact on humans by t...,"\nThe rise of e-health, or the use of electron...",21,8
2,2,4321.0,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",43,20
3,3,432.0,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",43,20
4,4,2893.8,Rise of Chatbots and its impact on customer su...,\nThe human race is known to come up with inve...,23,8


In [ ]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [ ]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/(  np.array(words_cleaned) + 0.000001)

In [ ]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [ ]:
complex_words = []
sylabble_counts = []

In [ ]:
for article in df['Contents']:
  sylabble_count=0
  d=article.split(' ')
  ans=0
  for word in d:
    count=0
    for i in range(len(word)):
      if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
           count+=1
#            print(words[i])
      if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
        count-=1;
      if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
        count-=1;
    sylabble_count+=count
    if(count>2):
        ans+=1
  sylabble_counts.append(sylabble_count)
  complex_words.append(ans)

In [ ]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

In [ ]:
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])

In [ ]:
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']

In [ ]:
df['COMPLEX WORD COUNT'] = complex_words

In [ ]:
df['WORD COUNT'] = words

In [ ]:
df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)

In [ ]:
total_characters = []
for article in df['Contents']:
  characters = 0
  for word in article.split():
    characters+=len(word)
  total_characters.append(characters)

In [ ]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And']
for article in df['Contents']:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)

In [ ]:
df['PERSONAL PRONOUN'] = personal_nouns

In [ ]:
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

In [ ]:
df.tail()

,Unnamed: 0,URL_ID,Title,Contents,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
97,97,50921.0,Coronavirus: Impact on the Hospitality Industry,\nBefore jumping on the topic I would like to ...,11,27,-0.421053,0.093596,25.500000,0.232680,10.293072,25.500000,178,765,1.677124,8,4.828758
98,98,51382.8,Coronavirus impact on energy markets,\nAs the coronavirus spreads around the world ...,32,57,-0.280899,0.082331,37.740000,0.218336,15.183334,37.740000,412,1887,1.697403,33,4.727080
99,99,51844.6,What are the key policies that will mitigate t...,"\nFrom Alibaba to Ping An and Google to Ford, ...",75,19,0.595745,0.089866,28.014286,0.242733,11.302808,28.014286,476,1961,1.707802,16,4.773075
100,100,52306.4,Marketing Drives Results With A Focus On Problems,"\nWhen\nthe British ruled India, many Indians\...",27,13,0.350000,0.048960,26.779661,0.222152,10.800725,26.779661,351,1580,1.639241,16,4.639241
101,101,52768.2,Continued Demand for Sustainability,\nThe business of business is no longer to do ...,18,23,-0.121951,0.065600,27.487805,0.307897,11.118281,27.487805,347,1127,1.931677,13,5.261757


In [ ]:
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
df=df.drop('Contents',axis=1)

In [ ]:
df.head()

,URL_ID,Title,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,123.0,Rise of telemedicine and its Impact on Livelih...,84,15,0.696970,0.098507,23.125000,0.309189,9.373676,23.125000,572,1850,1.964324,15,5.141081
1,321.0,Rise of e-health and its impact on humans by t...,21,8,0.448276,0.084058,26.560000,0.314759,10.749904,26.560000,209,664,1.954819,5,5.118976
2,4321.0,Rise of telemedicine and its Impact on Livelih...,43,20,0.365079,0.083113,22.900000,0.280932,9.272373,22.900000,386,1374,1.827511,22,5.053130
3,432.0,Rise of telemedicine and its Impact on Livelih...,43,20,0.365079,0.083113,22.900000,0.280932,9.272373,22.900000,386,1374,1.827511,22,5.053130
4,2893.8,Rise of Chatbots and its impact on customer su...,23,8,0.483871,0.044798,20.030769,0.267281,8.119220,20.030769,348,1302,1.783410,22,4.996160


In [ ]:
df.to_csv('Output Data Structure.csv')